# Here we will go over featureCounts in a little more detail 

FeatureCounts is a program that is a part of a larger software package called Subread, which collectively aligns and quantifies total read information. This is great, but because we have already aligned (and sorted) our reads, we are just interested in using featureCounts. FeatureCounts is an easy to process and efficient (summarize 20M reads in 0.5 min) process that allows us to quantify our data at a gene and strand-specific level.


**1) Installs**

Please perform the necessary installs, found in 6)FeatureCounts.ipynb. 

*Remember: the best method to install a program is through Anaconda. How do we do this again?* 



**2) Documentation**

Let's now go to the [documentation](http://bioinf.wehi.edu.au/subread-package/SubreadUsersGuide.pdf) and take a look at some of the details of featureCounts.

As you can see, featureCounts is program that is part of a larger program called Subread/RSubread. RSubread is a version of Subread designed to be operated in R, as opposed to Subread which is operated through Linux. We'll be using the Linux version for all of our processing.


*What are we actually doing with featureCounts?*

In using featureCounts to summarize our gene expression data. So far, we have managed to sequence (Fastq), align and filter (SAM/BAM), and sort our RNA-seq reads, but they still don't mean much to us in their present form. We need to take that data that we have from our individual samples and assign genomic features to our sequencing reads in a process we call *gene quantification.* In doing so, we will give featureCounts our mapped and sorted sequence fragments and provide us with an organized count table summarizing our expression data on a per gene basis. Here, we will have a record of all reads reported for each gene in each library. 


*What does featureCounts need from us to work?*

According to the documentation, featureCounts only needs: 1) aligned reads on either SAM or BAM format and 2) a list of genomic features in a GTF, General Feature Format (GFF) or Simplified Annotation Format (SAF) file. The annotation file must match the reference genome that the reads were mapped to. Luckily, we still have our hand GTF file available to use for this counting step. 


*Features or meta-features?*

FeatureCounts gives us flexibility to summarize our data by feature (e.g. exons) or by a collection of features, known as meta-features (e.g. genes). Since we are interested in differential gene expression, which would include the expression information of ALL mRNA isoforms of a particular gene, we will be looking on a meta-feature basis.


Of course this is a simplified explanation of what featureCounts can do. Please feel free to read the manual to see the full range of what featureCounts can do for sequencing experiments.  




# Using the featureCounts program

Before we begin, let's make a file in our project's directory for our outputs file:

    
    mkdir ~/projects/fto_shrna/featurecounts
    

We can then move on to writing a shell script for featureCounts in our scripts directory. The syntax of the featureCounts command is fairly simple:

    featureCounts -o counts.txt input files
    
    
But, there are A LOT of parameters/flags that can be included when making your final command. Let's look at a few important ones:


***Input files and output files***

Arguably the most important feature, you need to provide featureCounts with the files you want to be counted! FeatureCounts supports both SAM and BAM format, and will automatically detect file type upon submission. It also allows for multiple files to be submitted together.


We also need to give it a name and location for out output file, which will consist of our total gene counts in a standard text (.txt) format. Therefore, our argument looks something like this:  

    featureCounts -o featureCounts.txt library_1.bam library_2.bam library_3.bam library_4.bam
    
    
***Annotation file***

In order to provide gene quantification, we need to give the program an annotation file with the flag *-a*. We have our GTF file that we used with our alignment, and so we can provide that:


    featureCounts -a ~/scratch/annotations/hg19/gencode.v19.annotation.gtf -o featureCounts.txt library_1.bam library_2.bam library_3.bam library_4.bam
    
    
According to the manual, this is technically all we need to make the program work. But, let's look a little further for some other arguments to consider.


***How many threads should we run this job on?***

Computationally, this is a fairly fast and light program to run; we probably don't need to dedicate more than 2 cores/threads to complete this job. So, using the *-T* argument, we can include -T 2 to say that our job will run on 2 threads. The default is 1. 


***What else should be consider?***

*Is out data from a single-end or pared-end RNA seq experiment?* 
If it's from a paired-end experiment, we probably want to make it so both read1 and read2 of each sequenced fragment align before being called. To do so, we use the argument -B. It also states that -B is meant to be used to the -p flag, which indicates that this came from a paired-end experiment. Let's include both -B and -p.


*What features are we considering?* 

Because we are defining features from a pre-annotated GTF file, we have to provide the program with a means to quantify our reads. This means that only rows that match a defined feature type provided within the GTF file will be counted by featureCounts. We can consider a number of features within our counting, including exons, untranslated regions/UTRs, start_codons, etc. Because we are interested in looking at gene expression from coding genes, we can include *-t exon* as our argument. However, because exon is the default feature, we can skip it.


*What do we want to group our data based on?* 

This means, when going through our aligned BAM files, how do we want to group our features in our final counts matrix? We are interested in looking at our read count on a per gene basis, and therefore it would make sense for us to use the *-g gene_id* argument. Again, because it is already the default value, we do not need to include this in our final command.



*Do our RNA-seq libraries have strand information?*
One thing that we covered in the library preparation section of the class was the principle of making a library that preserves strandedness, or keeping track of which strand of DNA that individual reads were transcribed from. This method is very useful, as we would otherwise lose this information during the PCR amplification step. We can note this information in our command with the argument -s.

As you might remember, our libraries were made *reversely-stranded,* meaning that the reads that were sequenced were a reverse-complement of the original RNA/cDNA transcript. Seeing that this is the case, we will use the argument -s 2 to indicate that our libraries are reversely-stranded.


**Submitting our final job:**

As said before, this is a fairly small job that can be run on the login node. However, it's generally a good idea to write a shell script for your commands. This will give you a reference point if you want to run featureCounts again with a future dataset, and will prevent you from having to write out the command every single time. Our final command will look something like this. 
 
 
    #!/bin/bash
    #PBS -q hotel
    #PBS -N featurecounts
    #PBS -l nodes=1:ppn=2
    #PBS -l walltime=1:00:00
    #PBS -o featre_out
    #PBS -e feature_err

    featureCounts -a ~/scratch/annotations/hg19/gencode.v19.annotation.gtf -B -t exon -T 2 -o ~/projects/fto_shrna/featurecounts/featureCounts.txt -p -s 2 ~/biom200/all_bams/*.bam   

Notice that I use a wildcard character ( * ) to call all of my BAM files at once (*.bam). Because the program's output is one counts table, and because the syntax allows submitting multiple input files together (1.bam 2.bam 3.bam), we can use a * for this application. This will not always be the case for all programs, but we'll use it here to prevent ourselves from having to enter each name separately.


Also notice that my parameters may be sligtly different from those that we decided to use in class. That will probably most always be the case with data analysis: different people take into account different information. As long as you are being thoughtful in your analysis are being transparent in your analytical process, this is completely fine. In fact, your final counts table will more than likely look different than mine due to different parameters introduced at each level of our pipeline. 


Feel free to try a lot of different parameters and compare how your final outputs look each time!